This notebook will be collected automatically at **6pm on Monday** from `/home/data_scientist/assignments/Week14` directory on the course JupyterHub server. If you work on this assignment on the course Jupyterhub server, just make sure that you save your work and instructors will pull your notebooks automatically after the deadline. If you work on this assignment locally, the only way to submit assignments is via Jupyterhub, and you have to place the notebook file in the correct directory with the correct file name before the deadline.

1. Make sure everything runs as expected. First, restart the kernel (in the menubar, select `Kernel` → `Restart`) and then run all cells (in the menubar, select `Cell` → `Run All`).
2. Make sure you fill in any place that says `YOUR CODE HERE`. Do not write your answer in anywhere else other than where it says `YOUR CODE HERE`. Anything you write anywhere else will be removed by the autograder.
3. Do not change the file path or the file name of this notebook.
4. Make sure that you save your work (in the menubar, select `File` → `Save and CheckPoint`)

## Problem 14.2. Spark DataFrames

In this problem, we will use the Spark DataFrame to perform basic data processing tasks.

In [ ]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
import pandas as pd

from nose.tools import assert_equal, assert_is_instance
from pandas.util.testing import assert_frame_equal

We run Spark in [local mode](http://spark.apache.org/docs/latest/programming-guide.html#local-vs-cluster-modes) from within our Docker container.

In [ ]:
sc = SparkContext('local[*]')

We create a new RDD by reading in the data as a textfile. We use the ratings data from [MovieLens](http://grouplens.org/datasets/movielens/latest/). See [Week 6 Lesson 1](https://github.com/UI-DataScience/info490-sp16/blob/master/Week6/notebooks/intro2rs.ipynb) for more information on this data set.

In [ ]:
csv_path = '/home/data_scientist/data/ml-latest-small/ratings.csv'
text_file = sc.textFile(csv_path)

- Write a function that creates a Spark DataFrame from `text_file`. For example, running

```python
>>> df = create_df(text_file)
>>> df.show()
```

should give

```
+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     16|   4.0|1217897793|
|     1|     24|   1.5|1217895807|
|     1|     32|   4.0|1217896246|
|     1|     47|   4.0|1217896556|
|     1|     50|   4.0|1217896523|
|     1|    110|   4.0|1217896150|
|     1|    150|   3.0|1217895940|
|     1|    161|   4.0|1217897864|
|     1|    165|   3.0|1217897135|
|     1|    204|   0.5|1217895786|
|     1|    223|   4.0|1217897795|
|     1|    256|   0.5|1217895764|
|     1|    260|   4.5|1217895864|
|     1|    261|   1.5|1217895750|
|     1|    277|   0.5|1217895772|
|     1|    296|   4.0|1217896125|
|     1|    318|   4.0|1217895860|
|     1|    349|   4.5|1217897058|
|     1|    356|   3.0|1217896231|
|     1|    377|   2.5|1217896373|
+------+-------+------+----------+
only showing top 20 rows
```

In [ ]:
def create_df(rdd):
    '''
    Parameters
    ----------
    rdd: A pyspark.rdd.RDD instance.
    
    Returns
    -------
    A pyspark.sql.dataframe.DataFrame instance.
    '''
    
    # YOUR CODE HERE
    
    return df

In [ ]:
df = create_df(text_file)
df.show()

In [ ]:
assert_is_instance(df, pyspark.sql.dataframe.DataFrame)

# we the Spark dataframe to Pandas dataframe
df_pd = pd.read_csv(csv_path)
assert_frame_equal(df.toPandas(), df_pd)

- Select from the Spark DataFrame only the rows whose rating is greater than 3.
- After filtering, return only two columns: `movieId` and `rating`.

```python
>>> favorable = filter_favorable_ratings(df)
>>> favorable.show()
```

```
+-------+------+
|movieId|rating|
+-------+------+
|     16|   4.0|
|     32|   4.0|
|     47|   4.0|
|     50|   4.0|
|    110|   4.0|
|    161|   4.0|
|    223|   4.0|
|    260|   4.5|
|    296|   4.0|
|    318|   4.0|
|    349|   4.5|
|    457|   4.0|
|    480|   3.5|
|    527|   4.5|
|    589|   3.5|
|    590|   3.5|
|    593|   5.0|
|    608|   3.5|
|    648|   3.5|
|    724|   3.5|
+-------+------+
only showing top 20 rows
```

In [ ]:
def filter_favorable_ratings(df):
    '''
    Selects rows whose rating is greater than 3.
    
    Parameters
    ----------
    A pyspark.sql.dataframe.DataFrame instance.

    Returns
    -------
    A pyspark.sql.dataframe.DataFrame instance.

    '''
    
    # YOUR CODE HERE
    
    return df

In [ ]:
favorable = filter_favorable_ratings(df)
favorable.show()

In [ ]:
assert_is_instance(favorable, pyspark.sql.dataframe.DataFrame)

favorable_pd = df_pd.loc[df_pd['rating'] > 3.0, ['movieId', 'rating']].reset_index(drop=True)
assert_frame_equal(favorable.toPandas(), favorable_pd)

- Write a function that, given a `movieId`, computes the number of reviews for that movie.

In [ ]:
def find_n_reviews(df, movie_id):
    '''
    Finds the number of reviews for a movie.
    
    Parameters
    ----------
    movie_id: An int.
    
    Returns
    -------
    '''
    
    # YOUR CODE HERE
    
    return n_reviews

In [ ]:
n_toy_story = find_n_reviews(favorable, 1)
print(n_toy_story)

In [ ]:
assert_is_instance(n_toy_story, int)

test = [find_n_reviews(favorable, n) for n in range(1, 6)]
test_pd = favorable_pd.groupby('movieId').size()[:5].tolist()
assert_equal(test, test_pd)

## Cleanup

We must stop the SparkContext in order to release the spark resources before existing this Notebook.

In [ ]:
sc.stop()